In [104]:
import numpy as np
import pandas as pd
import time as t
from pdfminer.high_level import extract_text


df = pd.read_excel(r"L:\PRIV\ToxValDB\Document Mapping\0 - To Do\special_consideration\source_pprtv_ornl\pprtv_ornl_document_map_08032022_mmille16.xlsx")
#First, make additional columns for document map fields
string = ["" for x in range (len(df))]
df['chemical_name'] = string
df['casrn'] = string
df['source_name'] = string
df['document_name'] = string
df['clowder_id'] = string


In [105]:
#Now that our dataframe is initialized with all necessary columns, lets fill in the easy stuff first.
#Some info is a repeat of what appeared when the clowder log was first made or when metadata file was created.
#These fields are the source_name, document_name, and clowder_id fields.
#Just copy paste them over into the appropriate columns.

for i in range(3,len(df)):
    df.loc[i,'document_name'] = df.loc[i,'File Name'] #Added metadata
    df.loc[i,'clowder_id'] = df.loc[i,'uuid'] #Original from clowder log creation
    df.loc[i,'source_name'] = 'PPRTV ORNL' #Added metadata

In [106]:
#Next lets do the chemical names. I did this plenty of times when getting metadata so I will just copy over that script.
#Chemical names are in the file names. We just need to pull them out.
#For pdfs, chemical name is between 'ornl' and 'summary'
#For pngs, chemical name is between 'ornl' and 'webpage'

for i in range(3,len(df)):
    file_name = df.loc[i,'File Name']
    if '.pdf' in file_name:
        chem_start = file_name.index('ornl')+5
        chem_end = file_name.index('summary')-1
    if '.png' in file_name:
        chem_start = file_name.index('ornl')+5
        chem_end = file_name.index('webpage')-1
    chem_name = file_name[chem_start:chem_end]
    df.loc[i,'chemical_name'] = chem_name

In [ ]:
#Lastly we look for casrn numbers. We can use pdf text scraping for this since we have access to file names
#Typically casrns are located on the front page but some chemicals only have "various casrns.
#Some pdfs also pertain to multiple chemicals.
#Lets pick one that have several known casrns and see if we can write a script to accomplish this task.

#First, get the text. Then ignore if it has "various" or "multiple".
#Find the chemical name and all regex matches
#Look for a regex match after the chemical name but before the index of any additional matches
from pdfminer.high_level import extract_text
from pdfminer.high_level import extract_pages
import re

i = 238
#Name of file to open
file_name = df.loc[i,'File Name']
#Name of associated chemical
chemical_name = df.loc[i,'chemical_name']

#Some chemical names have their order switched. Not sure about chemical naming but this seemed to fix it some of the time
if chemical_name[-1] == '-':
    start = chemical_name[chemical_name.index(','):]
    end = chemical_name[0:-4]
    chemical_name = start+end


#Location of pdf files
file_path = "L:\\PRIV\\ToxValDB\\Document Webscrape\\pprtv_ornl\\output\\support_pdfs\\"

#Looking only at the first page of text with no extra white space nor new lines for the casrn numbers for now
text = extract_text(file_path+file_name, page_numbers = [0])
text = " ".join(text.split())
#print(chemical_name)
#Putting the text in all lower case to avoid capitalization issues in the future
search_text = text.lower()

#Find the index of the start of the chemical name
chem_index = search_text.index(chemical_name.lower())

#Initialize a counter to help us stop at the correct casrn number
counter = 0

#Find the location of all substrings of the form (numbers)-(numbers)-(numbers)
match = re.findall(r'\d*-\d*-\d*',text)
#print(match)


for string in match:
#Find the index of each match substring    
    loc = search_text.index(string)
#Once the index of the substring is larger than the index of our chemical, we can stop. We have found the chemicals casrn
    if loc > chem_index and counter == 0:
        counter += 1
        cas = string



In [ ]:
#Save the dataframe as an excel file
out_path = r"L:\PRIV\ToxValDB\Document Mapping\0 - To Do\special_consideration\source_pprtv_ornl\pprtv_ornl_document_map_in_progress_mmille16.xlsx"
with pd.ExcelWriter(out_path) as writer:
    df.to_excel(writer, index = False)

In [107]:
#Chemical names in pdf can vary quite a bit and I am not well versed enough in the chemistry to know the difference in naming
#Before doing too much work trying to fix problems, lets find the carns for the easy chemicals
#Lets, for now, ignore any papers that have "various" in their casrn section or have multiple casrns for multiple chemicals


for i in range(3,len(df)):
#Pull out the constant file name and chemical name and store. We will need these
    file_name = df.loc[i,'File Name']
    chemical_name = df.loc[i,'chemical_name']
    if '.png' in file_name: #Ignore pngs for now
        continue
#Extract the text from the support pdf cover page without newlines, white space, or capital letters.
    text = extract_text(file_path+file_name, page_numbers = [0])
    text = " ".join(text.split())
    text = text.lower()
    if 'various' in text or 'multiple' in text: #Ignore 'various' or 'multiple' casrn cover pages
        continue
    if text.count('casrn') > 1: #Ignore any papers with 'casrn' on the cover more than once
        continue
    if text.count('casrn') == 0: #If the cover does not have a casrn at all, thats its own deal
        print('casrn does not appear on the title page for '+ chemical_name)
        continue
#Find any strings of numbers that follow the pattern of a casrn. Note: this also pulls out the date
    match = re.findall(r'\d*-\d*-\d*',text)
#See if you can find the word casrn
    try:
        cas_index = text.index('casrn')
        counter = 0
        for string in match:
#Look through the matched number strings and see which one starts right after 'casrn' starts. That is the casrn we are looking for
            loc = text.index(string)
            if loc > cas_index and counter == 0:
                counter += 1
                casrn = string
#Fill in the casrn section of the dataframe
        df.loc[i,'casrn'] = casrn
    except:
#Take stock of the chemical names we didn't find matches for but keep the list going
        print('something went wrong finding the casrn for '+ chemical_name)
        continue

casrn does not appear on the title page for Hexane, Commercial
casrn does not appear on the title page for Methylcyclohexylamine, n-
casrn does not appear on the title page for Midrange Aliphatic Hydrocarbon Streams
casrn does not appear on the title page for Thallium (I) Nitrate
casrn does not appear on the title page for Thallium (Soluble Salts)
casrn does not appear on the title page for Thallium Acetate
casrn does not appear on the title page for Thallium Carbonate
casrn does not appear on the title page for Thallium Chloride
casrn does not appear on the title page for Thallium Sulfate


In [124]:
#Sometimes the word 'casrn' isn't there, but the casrns are listed after the chemical names.
#This was common when groups of chemicals shared a pdf.
#Now lets look at the ones we didn't find the easy carns for. Maybe they were a part of a group of chemicals sharing a support pdf file
#Next step is to look for the casrn number pattern immediately after a chemical name
#To do that: find the index of a chemical name, find the indicies of casrns, then pick the casrn immediately after the chemical name index


for i in range(3,len(df)):
    if df.loc[i,'casrn'] == "": #Only look for the ones we haven't found casrns for. Don't undo the work we just did
#Again, we need file and chemical names. Extract them and save them for later
        file_name = df.loc[i,'File Name']
        chemical_name = df.loc[i,'chemical_name']
        if '.png' in file_name: #Ignore .pngs for now.
            continue
#Extract the text from the first page of the support pdf with no newlines, whitespace, or capital letters
        text = extract_text(file_path+file_name,page_numbers = [0])
        text = " ".join(text.split())
        text = text.lower()
        if 'various' in text or 'multiple' in text: #Ignore if the cover has 'various' or 'multiple' casrn
            continue
#Find strings of numbers that match that of a casrn. Note: will consider the date as a match.
        match = re.findall(r'\d*-\d*-\d*',text)
        counter = 0
        try:
#Now try looking for the name of the chemical instead of the word 'casrn'
            chem_index = text.index(chemical_name.lower())
            for string in match:
#Same deal as above. Look through the list of matches for their starting indicies and stop once the start index > that of the chemical name
                loc = text.index(string)
                if loc > chem_index and counter == 0:
                    counter += 1 
                    casrn = string
#Fill in the casrn section of the dataframe
            df.loc[i,'casrn'] = casrn
        except:
#Keep track of exceptions
            print('could not find '+chemical_name+' on first page of support pdf')
            continue
            
        

could not find Ammonium polyphosphate (APP) on first page of support pdf
could not find Dichloro-2-butene, 1,4- on first page of support pdf
could not find Dichloro-2-butene, cis-1,4- on first page of support pdf
could not find Dichloro-2-butene, trans-1,4- on first page of support pdf
could not find Hexane, Commercial on first page of support pdf
could not find Methoxyethanol Acetate, 2- on first page of support pdf
could not find Methoxyethanol, 2- on first page of support pdf
could not find Methylcyclohexylamine, n- on first page of support pdf
could not find Perfluorobutanesulfonic acid (PFBS) on first page of support pdf
could not find Propylene Glycol Monoethyl Ether, alpha- on first page of support pdf
could not find Propylene Glycol Monoethyl Ether, beta- on first page of support pdf
could not find Samarium Chloride (Stable, Nonradioactive) on first page of support pdf
could not find Samarium Nitrate (Stable, Nonradioactive) on first page of support pdf
could not find Thallium 

In [126]:
#Another issue is the order of some chemicals with commas in the name.
    #Ex: Dichloro-2-butene,1,4- appears instead as 1,4-dichloro-2-butene in the paper
#I don't know enough chemistry to know the official rule for these naming conventions but lets try just breaking the chemical name
#at the first comma and looking instead for the string of numbers to the right of that comma. That will indicate the start of the chemical name

for i in range(3,len(df)):
    if df.loc[i,'casrn'] == "": #Do not undo the work we already did
#Save the file and chemical names for future use
        file_name = df.loc[i,'File Name']
        chemical_name = df.loc[i,'chemical_name']
        if '.png' in file_name: #Skip .png files for now
            continue
        text = extract_text(file_path+file_name, page_numbers = [0])
        text = " ".join(text.split())
        text = text.lower()
        if 'various' in text or 'multiple' in text:
#The comma names seem to be the last of the problems that are best to automate. The rest will be done manually.
#Fill in 'various' for the 'various' casrn chemicals now so I don't have to do that manually but don't do anything else with them.
            df.loc[i,'casrn'] = 'Various'
            continue
#Look for strings of numbers that match the pattern of a casrn. Note: this also considers the date a match
        match = re.findall(r'\d*-\d*-\d*',text)
        counter = 0
        try:
#Break the chemical name at the first comma and save everything to the right. We will look for that string now.
            comma = chemical_name.index(',') + 1
            right = chemical_name[comma:]
            chem_index = text.index(right.lower())
            for string in match:
#Same process as used before. Look for where the start of a casrn number is and stop looking once that start is larger than the beginning of the chemical name
                loc = text.index(string)
                if loc > chem_index and counter == 0:
                    counter += 1
                    casrn = string
            df.loc[i,'casrn'] = casrn
        except:
#In looking through exceptions, some had greek words like 'alpha' or 'beta' to describe isomers. These did not follow the pattern above as strictly
#Instead, we look for the last word instead of everything to the right of the comma and match that to the cover page
    #Example of this was chemical name: Propylene Glycol Monoehtyl Ether, alpha-.
    #Appears on title page as: Propylene Glycol Monethyl Ether (alpha isomer *casrn* beta isomer *casrn*)
            try:
#Find the comma and look only for the last word to the right of that comma
                comma = chemical_name.index(',') + 1
                right = chemical_name[comma:]
                right_word = re.findall(r'\w+',right)[0]
#Find where that last word starts
                chem_index = text.index(right_word.lower())
                for string in match:
#Look through the list of casrn matches until its starting index is bigger than the starting index of the chemical
                    loc = text.index(string)
                    if loc > chem_index and counter == 0:
                        counter += 1
                        casrn = string
#Plug in the casrn to the dataframe
                df.loc[i,'casrn'] = casrn
            except:
#These are the final stragglers that I will find manually. Not many, only 7 chemicals.
                print('could not find ' + chemical_name+ ' on the first page of support pdf')

could not find Ammonium polyphosphate (APP) on the first page of support pdf
could not find Perfluorobutanesulfonic acid (PFBS) on the first page of support pdf
could not find Thallium (Soluble Salts) on the first page of support pdf
could not find Thallium Acetate on the first page of support pdf
could not find Thallium Carbonate on the first page of support pdf
could not find Thallium Chloride on the first page of support pdf
could not find Thallium Sulfate on the first page of support pdf


In [127]:
#Look for where blank casrn numbers are (only looking at .pdfs since chemical names repeat for the .pngs)
blank = 0
for i in range(len(df)):
    file_name = df.loc[i,'File Name']
    if '.pdf' in str(file_name):
        if df.loc[i,'casrn'] == "":
            blank = blank + 1
print(blank)

7


In [128]:
chemical_names = []
casrns = []
#To make life a bit easier, I will save the chemical identifying information to a different excel sheet
#Remember, chemicals will appear twice in different locations. To save time: first, create a dataframe with the casrn and names I have found
#Then save the dataframe to an excel sheet
#Finally, write a script to match names and fill in casrns for me.
#This way, I fill in the identifier file, save it, and call it as a different dataframe for the computer to look through
#This also makes manual spot checking of casrn numebrs a bit easier.
for i in range(3,len(df)):
    file_name = df.loc[i,'File Name']
    if '.pdf' in file_name:
        chemical_names.append(df.loc[i,'chemical_name'])
        casrns.append(df.loc[i,'casrn'])
d = {'chemical_names':chemical_names, 'casrn': casrns}        
chemical_ids = pd.DataFrame(d)

In [130]:
#Write an excel file for the chemical casrn key. This is for easier spot checking of casrn numbers and to assit manual
#filling of carns
out_path = r"L:\PRIV\ToxValDB\Document Mapping\0 - To Do\special_consideration\source_pprtv_ornl\chemical_ids_key.xlsx"
with pd.ExcelWriter(out_path) as writer:
    chemical_ids.to_excel(writer, index = False)

In [134]:
#Fill in the casrns from the key we just made
key = pd.read_excel(r"L:\PRIV\ToxValDB\Document Mapping\0 - To Do\special_consideration\source_pprtv_ornl\chemical_ids_key.xlsx")
for i in range (3,len(df)):
    chemical_name = df.loc[i,'chemical_name']
    place = key[key['chemical_names'] == chemical_name].index.values
    df.loc[i,'casrn'] = key.loc[place[0],'casrn']

In [138]:
#Check to see where there are blank casrn numbers (if any)
blanks = 0
for i in range(3,len(df)):
    if df.loc[i,'casrn'] == "":
        print(df.loc[i,'chemical_name'])
        blanks += 1
print(blanks)

0


In [139]:
#Save the final dataframe as the excel document map file. 
out_path = r"L:\PRIV\ToxValDB\Document Mapping\0 - To Do\special_consideration\source_pprtv_ornl\pprtv_ornl_document_map_08082022_mmille16.xlsx"
with pd.ExcelWriter(out_path) as writer:
    df.to_excel(writer, index = False)
    
    

    
    
#Cells used for testing and brainstorming are below

Cells for brainstorming and testing since using new packages like pdfminer six

In [122]:
#Testing text extraction from pdfminer six 
#Testing regex pattern recognition

i = 22

chemical_name = df.loc[i,'chemical_name']
file_name = df.loc[i,'File Name']
print(chemical_name)
text = extract_text(file_path+file_name, page_numbers = [0])
text = " ".join(text.split())
text = text.lower()

print(text)
print(chemical_name.lower())

text.index(chemical_name.lower())
#comma = chemical_name.index(',') + 1
#right = chemical_name[comma:]
#re.findall(r'\w+',right)[0]

Ammonium Picrate
epa/690/r-20/008f | september 2020 | final provisional peer-reviewed toxicity values for picric acid (2,4,6-trinitrophenol) (casrn 88-89-1) and ammonium picrate (casrn 131-74-8) u.s. epa office of research and development center for public health and environmental assessment
ammonium picrate


144

In [133]:
#Testing section of code to find the index of a string in a dataframe

i = 134
key = pd.read_excel(r"L:\PRIV\ToxValDB\Document Mapping\0 - To Do\special_consideration\source_pprtv_ornl\chemical_ids_key.xlsx")
chemical_name = df.loc[i,'chemical_name']
place = key[key['chemical_names'] == chemical_name].index.values

place[0]

131

In [ ]:
#Brainstorming solutions to problematic chemical names

i = 238
#Name of file to open
file_name = df.loc[i,'File Name']
#Name of associated chemical
chemical_name = df.loc[i,'chemical_name']

#Some chemical names have their order switched. Not sure about chemical naming but this seemed to fix it some of the time
if chemical_name[-1] == '-':
    comma = chemical_name.index(',')+1
    start = chemical_name[comma:]
    end = chemical_name[0:comma-1]
    chemical_name = start+end
print(chemical_name)